In [52]:
path = "/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/pictures_tensor/low_rank_tensor_approx/low-rank-tensor-approximation/src/"
include(path*"cumulants.jl")
include(path*"read_pictures.jl")
include(path*"moment_calc.jl")
include(path*"random.jl")

generate_random_gauss (generic function with 1 method)

In [1]:
rmprocs(workers())
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [63]:
cov_calc(data,n)

(
4x3 Array{Float64,2}:
 0.0120731   0.0120737   -4.75624e-5
 0.00587742  0.0058777   -4.75624e-5
 0.00393397  0.00393416  -4.75624e-5
 1.23751e-5  1.23757e-5  -4.75624e-5,

4x4 Array{Float64,2}:
 -0.0875626  -0.99471     0.0536524  -0.00245832
 -0.947935    0.0666331  -0.311292    0.00875608
  0.293493   -0.0746559  -0.917015   -0.259562  
  0.0872589  -0.0232028  -0.243521    0.965684  ,

[0.012073084210579177,0.005877424539753128,0.003933973270769709,1.2375084166562841e-5])

In [4]:
data, n = generate_random_gauss(5, 1000000);

In [53]:
data, n = read_hyperspectral("indian_pines_hyperspectral.npy", 60);
data = data/maximum(data);

In [54]:
C2 = get_cumulant2(data)

4x4 Array{Float64,2}:
  0.0059193     0.000546848  -6.7347e-5   -8.02316e-6 
  0.000546848   0.0112559    -0.00226516  -0.000709397
 -6.7347e-5    -0.00226516    0.00438169   0.00119477 
 -8.02316e-6   -0.000709397   0.00119477   0.000339925

In [39]:
@time C3 = get_cumulant3(data);

  0.779872 seconds (13.91 k allocations: 3.587 MB)


In [40]:
@time C4 = get_cumulant4(data);

  0.327761 seconds (28.13 k allocations: 2.185 MB)


In [62]:
# kurtosis
@time A,B,C = kurtosis_calc(data, n)
A

4x3 Array{Float64,2}:
 -1.11367   -1.11367   -3.15818e-12
  1.25494    1.25494   -1.92884e-11
 -0.700002  -0.700002  -1.27042e-11
  0.61275    0.61275   -7.35198e-8 

3.085492 seconds (3.54 M allocations: 164.673 MB, 1.18% gc time)


In [59]:
using MAT
matwrite("cumulants.mat", Dict{Any,Any}(
    "C2" => C2,
    "C3" => C3,
    "C4" => C4
))

In [32]:
using MAT
matwrite("cumulants.mat", Dict{Any,Any}(
    "C2" => C2,
    "C3" => C3
))

In [61]:
using Tensors
(norm(vec(C4), 2)^2)/2


1.0861860890382604e-6

In [25]:
using PyCall

In [26]:
unshift!(PyVector(pyimport("sys")["path"]), "")
@pyimport cross_cumulant as cc

3x3 Array{Float64,2}:
 0.431195  0.328736  0.180378
 0.328736  0.254933  0.139698
 0.180378  0.139698  0.077555

In [75]:
cov(data)

3x3 Array{Float64,2}:
  0.00612226   0.000111728  -6.69157e-6 
  0.000111728  0.000873043   0.000242232
 -6.69157e-6   0.000242232   0.00100628 

In [41]:
data1 = data - repmat(mean(data,1), size(data,1))

21025x4 Array{Float64,2}:
  0.0465286   -0.0134801     0.0448405   0.0201421 
 -0.0826715   -0.0221512     0.0387707   0.0177575 
 -0.0462526   -0.0438291     0.0342184   0.0160233 
  0.00924269  -0.060521      0.0305331   0.0171072 
 -0.044952    -0.0160814     0.0441902   0.0201421 
 -0.0826715    0.000177001   0.0526445   0.0216595 
  0.0465286    0.00516291    0.0402882   0.0184078 
  0.0465286   -0.00264112    0.0151418   0.00908635
 -0.0835386   -0.00155723   -0.0277803  -0.00717205
 -0.0828882   -0.125988     -0.0342837  -0.0112908 
 -0.085056     0.0032119    -0.0470736  -0.0130251 
 -0.048854     0.0147012    -0.0537938  -0.0130251 
 -0.0501547   -0.113415     -0.0579126  -0.0138922 
  ⋮                                                
  0.00447356  -0.13661      -0.0964992  -0.0223466 
  0.0783951    0.210019     -0.0663669  -0.0197452 
  0.042193     0.159293     -0.0852267  -0.0208291 
  0.00685812   0.0940422    -0.0891287  -0.0225633 
  0.153617    -0.0910868    -0.0908629

In [55]:
cc.compute_cumulant2(data1')-C2

4x4 Array{Float64,2}:
  4.77049e-17   0.0          -4.06576e-20  -9.99499e-20
  0.0          -3.1225e-17    1.30104e-18   0.0        
 -4.06576e-20   1.30104e-18  -2.77556e-17   8.67362e-18
 -9.99499e-20   0.0           8.67362e-18  -1.1872e-17 

In [49]:
cc.compute_cumulant3(data1')-C3

4x4x4 Array{Float64,3}:
[:, :, 1] =
 -9.21572e-19  -1.76183e-19   5.42101e-19  -2.57498e-19
 -1.76183e-19   7.45389e-20  -7.87741e-20  -2.5411e-20 
  5.42101e-19  -7.87741e-20  -2.10064e-19  -1.52466e-20
 -2.57498e-19  -2.5411e-20   -1.52466e-20  -5.50571e-21

[:, :, 2] =
 -1.76183e-19   6.09864e-20  -7.6233e-20   -2.5411e-20 
  6.09864e-20  -1.24683e-18  -6.77626e-19   2.50722e-19
 -7.6233e-20   -6.77626e-19   2.91379e-19   1.35525e-20
 -2.5411e-20    2.50722e-19   1.35525e-20   2.96462e-21

[:, :, 3] =
  5.42101e-19  -7.11508e-20  -2.10064e-19  -1.86347e-20
 -7.11508e-20  -6.50521e-19   2.91379e-19   1.35525e-20
 -2.10064e-19   2.91379e-19  -2.57498e-19  -1.77877e-19
 -1.86347e-20   1.35525e-20  -1.77877e-19   1.69407e-20

[:, :, 4] =
 -2.57498e-19  -2.75286e-20  -2.03288e-20  -5.50571e-21
 -2.75286e-20   2.43945e-19   1.18585e-20   2.11758e-21
 -2.03288e-20   1.18585e-20  -1.72795e-19   1.35525e-20
 -5.50571e-21   2.11758e-21   1.35525e-20   9.97381e-20

In [61]:
cc.compute_cumulant4(data1')

4x4x4x4 Array{Float64,4}:
[:, :, 1, 1] =
 -1.95927e-6  -1.43287e-5  2.68036e-5  7.52958e-6
 -1.43287e-5  -6.21784e-6  5.05537e-6  1.43433e-6
  2.68036e-5   5.05537e-6  3.69552e-6  1.2116e-6 
  7.52958e-6   1.43433e-6  1.2116e-6   3.88603e-7

[:, :, 2, 1] =
 -1.43287e-5  -6.21784e-6  5.05537e-6  1.43433e-6
 -6.21784e-6  -6.01859e-6  2.6497e-6   8.66965e-7
  5.05537e-6   2.6497e-6   1.97013e-6  6.49061e-7
  1.43433e-6   8.66965e-7  6.49061e-7  2.02715e-7

[:, :, 3, 1] =
 2.68036e-5  5.05537e-6  3.69552e-6  1.2116e-6 
 5.05537e-6  2.6497e-6   1.97013e-6  6.49061e-7
 3.69552e-6  1.97013e-6  2.48358e-6  7.26541e-7
 1.2116e-6   6.49061e-7  7.26541e-7  2.11638e-7

[:, :, 4, 1] =
 7.52958e-6  1.43433e-6  1.2116e-6   3.88603e-7
 1.43433e-6  8.66965e-7  6.49061e-7  2.02715e-7
 1.2116e-6   6.49061e-7  7.26541e-7  2.11638e-7
 3.88603e-7  2.02715e-7  2.11638e-7  6.18527e-8

[:, :, 1, 2] =
 -1.43287e-5  -6.21784e-6  5.05537e-6  1.43433e-6
 -6.21784e-6  -6.01859e-6  2.6497e-6   8.66965e-7
  5.05537e-

In [50]:
?cov

search: 

```
cov(v1[, v2][, vardim=1, corrected=true, mean=nothing])
```

Compute the Pearson covariance between the vector(s) in `v1` and `v2`. Here, `v1` and `v2` can be either vectors or matrices.

This function accepts three keyword arguments:

  * `vardim`: the dimension of variables. When `vardim = 1`, variables are considered in columns while observations in rows; when `vardim = 2`, variables are in rows while observations in columns. By default, it is set to `1`.
  * `corrected`: whether to apply Bessel's correction (divide by `n-1` instead of `n`). By default, it is set to `true`.
  * `mean`: allow users to supply mean values that are known. By default, it is set to `nothing`, which indicates that the mean(s) are unknown, and the function will compute the mean. Users can use `mean=0` to indicate that the input data are centered, and hence there's no need to subtract the mean.

The size of the result depends on the size of `v1` and `v2`. When both `v1` and `v2` are vectors, it returns the covariance between them as a scalar. When either one is a matrix, it returns a covariance matrix of size `(n1, n2)`, where `n1` and `n2` are the numbers of slices in `v1` and `v2`, which depend on the setting of `vardim`.

Note: `v2` can be omitted, which indicates `v2 = v1`.


cov cov_calc vcov invcov conv conv2 convert autocov autocov! deconv



In [51]:
methods(cov)

# 10 methods for generic function "cov":
cov(x::DenseArray{T,2}, wv::StatsBase.WeightVec{W,Vec<:AbstractArray{T<:Real,1}}) at /home/krzysztof/.julia/v0.4/StatsBase/src/cov.jl:45
cov(x::AbstractArray{T,1}) at statistics.jl:292
cov(x::AbstractArray{T,2}) at statistics.jl:299
cov(x::AbstractArray{T,1}, y::AbstractArray{T,1}) at statistics.jl:306
cov(x::Union{AbstractArray{T,1},AbstractArray{T,2}}, y::Union{AbstractArray{T,1},AbstractArray{T,2}}) at statistics.jl:313
cov(d::Distributions.Dirichlet) at /home/krzysztof/.julia/v0.4/Distributions/src/multivariate/dirichlet.jl:57
cov(d::Distributions.Multinomial) at /home/krzysztof/.julia/v0.4/Distributions/src/multivariate/multinomial.jl:47
cov(d::Distributions.MvNormal{Cov<:PDMats.AbstractPDMat,Mean<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}) at /home/krzysztof/.julia/v0.4/Distributions/src/multivariate/mvnormal.jl:114
cov(d::Distributions.MvNormalCanon{P<:PDMats.AbstractPDMat,V<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}) at /home/krzysztof/.julia/v0.4/Distributions/src/multivariate/mvnormalcanon.jl:70
cov(d::Distributions.GenericMvTDist{Cov<:PDMats.AbstractPDMat}) at /home/krzysztof/.julia/v0.4/Distributions/src/multivariate/mvtdist.jl:78